In [ ]:
# Installation of ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
# do it for only one time
 
 
### Install Dash
!pip install dash==1.4.1 
!pip install dash-auth
!pip install dash-html-components==0.13.2
!pip install dash-core-components==0.39.0
!pip install dash-bootstrap-components
!pip install dash-table==3.1.7 
!pip install dash-daq==0.2.1
!pip install pyforest
!pip install quandl
!pip install fredapi
!pip install EIA_python
!pip install ffn

--2020-09-10 17:19:00--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.170.115.131, 34.230.212.40, 3.229.170.137, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.170.115.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  29.9MB/s    in 0.4s    

2020-09-10 17:19:00 (29.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 1.2MB 7.0MB/s 
     |████████████████████████████████| 2.1MB 16.1MB/s 
     |████████████████████████████████| 194kB 34.0MB/s 
     |████████████████████████████████| 1.8MB 35.1MB/s 
     |████████████████████████████████| 358kB 31.8MB/s 
  Created w

In [ ]:
# Generate .py file to run with flask
%%writefile Sarit_app.py
from pyforest import *
import datetime, pickle, copy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
 
import quandl, eia, ffn
 
plt.style.use('ggplot')
from statistics import variance 
from random import randint
import scipy as sp
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc, classification_report, accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix 
from xgboost import XGBClassifier, XGBRegressor, plot_importance
 
from fredapi import Fred
 
from termcolor import colored
from sklearn.model_selection import cross_val_score, KFold, cross_validate, train_test_split, TimeSeriesSplit
import xgboost as xgb import Input, Output
import dash_table
import dash_bootstrap_components as dbc
 
USERNAME_PASSWORD = {
    'Sarit': 'KSG@analytics2020',
    'admin':'admin'
}
app = dash.Dash(__name__)
suppress_callback_excimport cross_val_score, KFold, cross_validate, train_test_split, TimeSeriesSplit
import xgboost as xgb

import dash
import dash_auth
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencieseptions=True
app.title = 'Natural Gas'
 
auth = dash_auth.BasicAuth(
    app,
    USERNAME_PASSWORD
)
 
 
# Henry Hub Spot Price, Daily (Release Date: 3/11/2020; Next Release Date: 3/18/2020 )
print('\033[4mHenry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    hh_spot = pd.DataFrame(series_search)
    return hh_spot
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.RNGWHHD.W'
      hh_spot = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(hh_spot))
      return hh_spot;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
hh_spot = main()
hh_spot = hh_spot.rename({'Henry Hub Natural Gas Spot Price, Weekly (Dollars per Million Btu)': 'hh_spot'}, axis = 'columns')
hh_spot = hh_spot.reset_index()
hh_spot['index'] = pd.to_datetime(hh_spot['index'].str[:-3], format='%Y %m%d')
hh_spot['Date']= pd.to_datetime(hh_spot['index']) 
hh_spot.set_index('Date', inplace=True) # setting index column
hh_spot = hh_spot.loc['2000-01-01':,['hh_spot']] # setting date range
hh_spot = hh_spot.astype(float)
#print(hh_spot) 
print('\n')
 
import plotly.express as px
figures = px.line(x=hh_spot.index, y=hh_spot['hh_spot'], labels={'x':'Date', 'y':'Spot Price'})
figures.show()
 
# Natural Gas continuous contract
 
print('\033[4mNatural Gas continuous contract 1\033[0m')
NG = quandl.get("CHRIS/CME_NG1", authtoken="LSQpgUzwJRoF667ZpzyL") # natural gas continuous contract 1
NG = NG.loc['2000-01-01':,]
 
 
# SPOT/OPN/CLOSE/ADJ CLOSE/HIGH/LOW/VOLUME
 
print('\033[4mCurrent spot price\033[0m')
print(round(hh_spot,2).tail(1))
print('\n')
print('\033[4mOpen price\033[0m')
print(round(NG['Open'],2).tail(1))
print('\n')
print('\033[4mClose price\033[0m')
print(round(NG['Last'],2).tail(1))
print('\n')
print('\033[4mAdj Close price\033[0m')
print(round(NG['Settle'],2).tail(1))
print('\n')
print('\033[4mHigh price\033[0m')
print(round(NG['High'],2).tail(1))
print('\n')
print('\033[4mLow price\033[0m')
print(round(NG['Low'],2).tail(1))
print('\n')
print('\033[4mVolume\033[0m')
print(round(NG['Volume'],2).tail(1))
print('\n')
 
 
import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=NG.index,
                                     open=NG['Open'],
                                     high=NG['High'],
                                     low=NG['Low'],
                                     close=NG['Last'])])
fig.show()
print('\n')
 
 
 
# 4 WEEKS's Open/Close/High/Low PRICE
 
ng = round(NG[['Open', 'Last', 'High', 'Low']],2).copy()
ng = ng.reset_index()
ng.tail()
 
 
# STATISTICS 1
 
perf = NG['Last'].copy().calc_stats()
print('\n')
perf.display()
print('\n')
 
 
#%matplotlib inline 
 
#ffn.to_drawdown_series(NG['Last']).plot(figsize=(15,7),grid=True)
#print('\n')
 
#print('\033[4mStatistics\033[0m')
#print(perf.stats)
#print('\n')
 
# STATISTICS 2
 
# Calculate the daily percentage change which is daily return
#print('\033[1m' + 'daily percentage change' + '\033[1m')
daily_ret = NG['Last'].pct_change().fillna(0)
mean_return = daily_ret.mean()
return_stdev = daily_ret.std()
print('Average daily return %:', round((mean_return*100),2))
print('Average Volatility %:', round((return_stdev*100), 2))
print('\n')
 
 
print ('Annualized average return %:', round(((mean_return * 252)*100),2))
print('Annualized volatility %:', round(((return_stdev * np.sqrt(252))*100), 2))
print('\n')
 
print('\033[4mMaximum Drawdown\033[0m')
 
# Using a trailing 186 trading day window
window = 186
 
# Calculate the max drawdown in the past window days for each day in the series.
# using min_periods=1, we want to let the first 186 days data have an expanding window
roll_max = NG['Last'].rolling(min_periods=1, window=window).max()
 
# Calculate daily draw-down from rolling max
daily_drawdown = NG['Last']/roll_max - 1.0
print('\033[4mMaximum Daily Drawdown\033[0m')
print(daily_drawdown)
print('\n')
 
# Next we calculate the minimum (negative) daily drawdown in that window.
# using min_periods=1 to allow the expanding window
print('\033[4mMin Daily Drawdown\033[0m')
min_daily_drawdown = daily_drawdown.rolling(min_periods=1, window=window).min()
print(min_daily_drawdown)
print('\n')
 
# Plot the results
daily_drawdown.plot(figsize=(20,8))
min_daily_drawdown.plot()
#plt.title('Orange -> Daily running 186-day drawdown; Blue -> Max. experienced 186-day drawdown in the past year')
plt.title('Maximum daily draw-down')
plt.grid()
plt.show()
print('\n')
 
 
print('\033[4mSharpe of daily return %\033[0m')
sharpe_ratio = daily_ret.mean() / daily_ret.std()
Sharpe = round(sharpe_ratio,2)*100
print('Sharpe Ratio:', Sharpe)
print('\n')
 
print('\033[4mAverage Annual Sharpe %\033[0m')
Sharpe_an = round(((252**0.5) * sharpe_ratio),2)*100
print('Annualized Sharpe %:', Sharpe_an)
 
 
# STATISTICS 3
 
print('\033[4mProbability of +/-(1%); +/-(3%); +/-%(5) change in price (Data -> 2000- till date)\033[0m')
 
print ("The probability of price changes between 1%% and -1%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.01) & (daily_ret < 0.01)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 3%% and -3%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.03) & (daily_ret < 0.03)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 5%% and -5%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.05) & (daily_ret < 0.05)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes more than 5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret > 0.05].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes less than -5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret < -0.05].shape[0] / daily_ret.shape[0]))
print('\n')
 
print('\033[4mMinimum price [2000- till date]\033[0m')
print(round(NG['Last'].min(),2), NG['Last'].idxmin());
print('\033[4mMaximum price [2000- till date]\033[0m')
print(round(NG['Last'].max(),2), NG['Last'].idxmax());
print('\n')
 
print('\033[4mMinimum daily % return [2000- till date]\033[0m')
print(round(daily_ret.min(),2)*100, daily_ret.idxmin()); 
print('\033[4mMaximum daily % return [2000- till date]\033[0m')
print(round(daily_ret.max()*100, 2), daily_ret.idxmax());
print('\n')
 
 
 
# DATA PIPELINE
 
# Cushing, OK WTI Spot Price FOB, Daily
print('\033[4mWTI Spot Price FOB, Daily (Dollars per Barrel)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    wti = pd.DataFrame(series_search)
    return wti
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='PET.RWTC.D'
      wti = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(wti))
      return wti;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
wti = main()
wti = wti.rename({'Cushing, OK WTI Spot Price FOB, Daily (Dollars per Barrel)': 'wti'}, axis = 'columns')
wti = wti.reset_index()
wti['index'] = pd.to_datetime(wti['index'].str[:-3], format='%Y %m%d')
wti['Date']= pd.to_datetime(wti['index']) 
wti.set_index('Date', inplace=True) # setting index column
wti = wti.loc['2000-01-01':,['wti']] # setting date range
wti = wti.astype(float)
#print(wti) 
#print('\n')
 
## GDP data
print('\033[4mUS GDP Data\033[0m')
fred = Fred(api_key='59c798fcda5850ef874412d269fd2378')
gdp = fred.get_series_as_of_date('GDP', '2020-01-31')
gdp = gdp.rename({'date': 'Date', 'value': 'gdp'}, axis = 'columns')
gdp.set_index('Date', inplace=True) # setting index column
gdp = gdp.loc['2000-01-01':'2020-02-02',['gdp']] # setting date range
gdp.replace({pd.NaT: "0"}, inplace=True) # replacing NaT with 0
gdp = gdp.ffill().bfill()
gdp = gdp.reset_index().drop_duplicates('Date').set_index('Date') # dropping duplicate concurrence
gdp = pd.DataFrame(gdp)
gdp = gdp.astype(float)
gdp = gdp.resample('B').ffill()
gdp = gdp/4
#print(gdp)
#print('\n')
 
# U.S. Natural Gas Marketed Production, Monthly
print('\033[4mNatural Gas Marketed Production, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    prod = pd.DataFrame(series_search)
    return prod
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9050US2.M'
      prod = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(prod))
      return prod;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
prod = main()
prod = prod.rename({'U.S. Natural Gas Marketed Production, Monthly (Million Cubic Feet)': 'prod'}, axis = 'columns')
prod = prod.reset_index()
prod['Date']= pd.to_datetime(prod['index']) 
prod.set_index('Date', inplace=True) # setting index column
prod = prod.loc['2000-01-01':,['prod']] # setting date range
prod = prod.astype(float)
prod = prod.resample('B').ffill()
prod = prod/21
#print(prod)
#print('\n')
 
 
# U.S. Natural Gas Gross Withdrawals, Monthly
print('\033[4mNatural Gas Gross Withdrawals, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    withdrawals = pd.DataFrame(series_search)
    return withdrawals
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9010US2.M' 
      withdrawals = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(withdrawals))
      return withdrawals;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
withdrawals = main()
withdrawals = withdrawals.rename({'U.S. Natural Gas Gross Withdrawals, Monthly (Million Cubic Feet)': 'withdrawals'}, axis = 'columns')
withdrawals = withdrawals.reset_index()
withdrawals['Date']= pd.to_datetime(withdrawals['index']) 
withdrawals.set_index('Date', inplace=True) # setting index column
withdrawals = withdrawals.loc['2000-01-01':,['withdrawals']] # setting date range
withdrawals = withdrawals.astype(float)
withdrawals = withdrawals.resample('B').bfill().ffill()
withdrawals = withdrawals/21
#print(withdrawals)
#print('\n')
 
# U.S. Natural Gas Underground Storage Volume, Monthly
print('\033[4mNatural Gas Underground Storage Volume, Monthly\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    storage = pd.DataFrame(series_search)
    return storage
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N5030US2.M '
      storage = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(storage))
      return storage;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
storage = main()
storage = storage.rename({'U.S. Natural Gas Underground Storage Volume, Monthly (Million Cubic Feet)': 'storage'}, axis = 'columns')
storage = storage.reset_index()
storage['Date']= pd.to_datetime(storage['index']) 
storage.set_index('Date', inplace=True) # setting index column
storage = storage.loc['2000-01-01':,['storage']] # setting date range
stoarge = storage.astype(float)
storage = storage.resample('B').bfill().ffill()
storage = storage/21
#print(storage)
#print('\n')
 
# U.S. Natural Gas Exports, Monthly
print('\033[4mNatural Gas Exports, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    export = pd.DataFrame(series_search)
    return export
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9133US2.M'
      export = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(export))
      return export;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
export = main()
export = export.rename({'Liquefied U.S. Natural Gas Exports, Monthly (Million Cubic Feet)': 'export'}, axis = 'columns')
export = export.reset_index()
export['Date']= pd.to_datetime(export['index']) 
export.set_index('Date', inplace=True) # setting index column
export = export.loc['2000-01-01':,['export']] # setting date range
export = export.astype(float)
export = export.resample('B').bfill().ffill()
export = export/21
#print(export)
#print('\n')
 
# natural gas total consumption (monthly))
print('\033[4mNatural Gas Total Consumption, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    consumption = pd.DataFrame(series_search)
    return consumption
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9140US2.M'
      consumption = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(consumption))
      return consumption;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
consumption = main()
consumption = consumption.rename({'U.S. Natural Gas Total Consumption, Monthly (Million Cubic Feet)': 'consumption'}, axis = 'columns')
consumption = consumption.reset_index()
consumption['Date']= pd.to_datetime(consumption['index']) 
consumption.set_index('Date', inplace=True) # setting index column
consumption = consumption.loc['2000-01-01':,['consumption']] # setting date range
consumption = consumption.astype(float)
consumption = consumption.resample('B').ffill().bfill()
consumption = consumption/21
#print(consumption)
#print('\n')
 
# New York Harbor No. 2 Heating Oil Spot Price FOB, Monthly
print('\033[4mHeating Oil Spot Price FOB, Daily (Dollars per Gallon)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    heatingoil = pd.DataFrame(series_search)
    return heatingoil
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='PET.EER_EPD2F_PF4_Y35NY_DPG.D'
      heatingoil = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(heatingoil))
      return heatingoil;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
 
heatingoil = main()
heatingoil = heatingoil.rename({'New York Harbor No. 2 Heating Oil Spot Price FOB, Daily (Dollars per Gallon)': 'heatingoil'}, axis = 'columns')
heatingoil = heatingoil.reset_index()
heatingoil['index'] = pd.to_datetime(heatingoil['index'].str[:-3], format='%Y %m%d')
heatingoil['Date']= pd.to_datetime(heatingoil['index']) 
heatingoil.set_index('Date', inplace=True) # setting index column
heatingoil = heatingoil.loc['2000-01-01':,['heatingoil']] # setting date range
heatingoil = heatingoil.astype(float)
#print(heatingoil)
#print('\n')
 
# natural gas import dat
#print('\033[4mNatural Gas Imports, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    g_import = pd.DataFrame(series_search)
    return g_import
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9100US2.M'
      g_import = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(g_import))
      return g_import;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
g_import = main()
g_import = g_import.rename({'U.S. Natural Gas Imports, Monthly (Million Cubic Feet)': 'g_import'}, axis = 'columns')
g_import = g_import.reset_index()
g_import['Date']= pd.to_datetime(g_import['index']) 
g_import.set_index('Date', inplace=True) # setting index column
g_import = g_import.loc['2000-01-01':,['g_import']] # setting date range
g_import = g_import.astype(float)
g_import = g_import.resample('B').ffill().bfill()
g_import = g_import/21
#print(g_import)
#print('\n')
 
# U.S. Natural Gas Rotary Rigs in Operation, Monthly
print('\033[4mNatural Gas Rotary Rigs in Operation, Monthly (Number of Elements)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    drilling = pd.DataFrame(series_search)
    return drilling
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='PET.E_ERTRRG_XR0_NUS_C.M '
      drilling = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(drilling))
      return drilling;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
drilling = main()
drilling = drilling.rename({'U.S. Natural Gas Rotary Rigs in Operation, Monthly (Number of Elements)': 'drilling'}, axis = 'columns')
drilling = drilling.reset_index()
drilling['Date']= pd.to_datetime(drilling['index']) 
drilling.set_index('Date', inplace=True) # setting index column
drilling = drilling.loc['2000-01-01':,['drilling']] # setting date range
drilling = drilling.astype(float)
drilling = drilling.resample('B').ffill().bfill()
drilling = drilling/21
#print(drilling)
#print('\n')
 
# Heating Degree Days U.S. Average , Monthly
print('\033[4mHeating Degree Days U.S. Average , Monthly (total degree days)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    HDD_US = pd.DataFrame(series_search)
    return HDD_US
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='STEO.ZWHDPUS.M'
      HDD_US = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(HDD_US))
      return HDD_US;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
HDD_US = main()
HDD_US = HDD_US.rename({'Heating Degree Days U.S. Average , Monthly (total degree days)': 'HDD_US'}, axis = 'columns')
HDD_US = HDD_US.reset_index()
HDD_US['Date']= pd.to_datetime(HDD_US['index']) 
HDD_US.set_index('Date', inplace=True) # setting index column
HDD_US = HDD_US.loc['2000-01-01':,['HDD_US']] # setting date range
HDD_US = HDD_US.astype(float)
HDD_US = HDD_US.resample('B').ffill().bfill()
HDD_US = HDD_US/21
#print(HDD_US)
#print('\n')
 
# Cooling Degree-Days, United States, Monthly
print('\033[4mCooling Degree-Days, United States, Monthly (Number)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    CDD_US = pd.DataFrame(series_search)
    return CDD_US
 
def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='TOTAL.ZWCDPUS.M'
      CDD_US = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(CDD_US))
      return CDD_US;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
CDD_US = main()
CDD_US = CDD_US.rename({'Cooling Degree-Days, United States, Monthly (Number)': 'CDD_US'}, axis = 'columns')
CDD_US = CDD_US.reset_index()
CDD_US['Date']= pd.to_datetime(CDD_US['index']) 
CDD_US.set_index('Date', inplace=True) # setting index column
CDD_US = CDD_US.loc['2000-01-01':,['CDD_US']] # setting date range
CDD_US = CDD_US.astype(float)
CDD_US = CDD_US.resample('B').ffill().bfill()
CDD_US = CDD_US/21
#print(CDD_US)
#print('\n')
 
# SUPERVISED ML
 
# merging data frames
merge1 = NG.join(gdp, how='left').ffill().bfill()
merge2 = merge1.join(wti, how = 'left')
merge3 = merge2.join(withdrawals, how = 'left').ffill().bfill()
merge4 = merge3.join(storage, how = 'left').ffill().bfill()
merge5 = merge4.join(prod, how = 'left').ffill().bfill()
merge6 = merge5.join(consumption, how = 'left').ffill().bfill()
merge7 = merge6.join(heatingoil, how ='left').ffill().bfill()
merge8 = merge7.join(drilling, how = 'left').ffill().bfill()
merge9 = merge8.join(export, how = 'left').ffill().bfill()
merge10 = merge9.join(g_import, how = 'left').ffill().bfill()
merge11 = merge10.join(HDD_US, how = 'left').ffill().bfill()
merge12 = merge11.join(CDD_US, how = 'left').ffill().bfill()
 
# feature engineering
merge12['day_of_week'] = merge12.index.dayofweek
merge12['day_of_month'] = merge12.index.day
merge12['quarter'] = merge12.index.quarter
merge12['month'] = merge12.index.month
merge12['year'] = merge12.index.year
 
 
# Technical indicators
merge12['H-L'] = merge12['High'] - merge12['Low']
merge12['O-C'] = merge12['Last'] - merge12['Open']
merge12['daily_ret'] = merge12['Last'].pct_change()
merge12['volatility'] = merge12['daily_ret'].rolling(252).std()*(252**0.5)
merge12["vol_increment"] = merge12['Volume'].diff()
merge12["vol_rel_increment"] = merge12['Volume'].diff() / merge12['Volume']
 
df = merge12.apply(copy.deepcopy)
#print(merge13)
 
sma1 = 3
sma2 = 10
sma3 = 20
sma4 = 30
sma5 = 100
df["sma1"] = df['Settle'].shift(1).rolling(sma1).mean()
df["sma2"] = df['Settle'].shift(1).rolling(sma2).mean()
df['sma3'] = df['Settle'].shift(1).rolling(sma3).mean()
df['sma4'] = df['Settle'].shift(1).rolling(sma4).mean()
df['sma5'] = df['Settle'].shift(1).rolling(sma5).mean()
df['Std_dev']= df['Settle'].rolling(5).std()
 
df['ema_12'] = df['Settle'].ewm(span=10).mean()
df['ema_26'] = df['Settle'].ewm(span=26).mean()
df['ROC'] = ((df['Settle'] - df['Settle'].shift(5)) / (df['Settle'].shift(5)))*100
 
delta = df['Settle'].diff()
window = 14
up_days = delta.copy()
up_days[delta<=0]=0.0
down_days = abs(delta.copy())
down_days[delta>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
df['rsi'] = 100-100/(1+RS_up/RS_down)
df['macd'] = df['ema_12'] - df['ema_26']
df['Williams%R'] = (df['High'].max() - df['Last'])/(df['High'] - df['Low'].min()) * -100
df['t-2'] = df['Settle'].shift(2)
df['t-1'] = df['Settle'].shift(1)
width = 3
shifted = df['Settle'].shift(width - 1)
roll_window = shifted.rolling(window=width)
df['min'] = roll_window.min()
df['mean'] = roll_window.mean()
df['max'] = roll_window.max()
exp_window = df['Settle'].expanding()
df['min_exp'] = exp_window.min()
df['mean_exp'] = exp_window.mean()
df['max_exp'] = exp_window.max()
 
df = df.astype(float)
#df.info()
 
# BUY / SELL SIGNAL 
 
df['label'] = np.where(df['Open'].shift(-1) > df['Open'], 1, 0)
 
X = np.array(df.copy().drop(['label',
                             'Open',
                             'Settle',
                             'High',
                             'Low',
                             'Last',
                             'Volume',
                             'Previous Day Open Interest'], axis=1))
 
y = np.array(df['label']).copy()
 
tscv = TimeSeriesSplit()
#print(tscv)
TimeSeriesSplit(max_train_size=0.67, n_splits=5)
for train_index, test_index in tscv.split(X):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
 
  # convert the numpy arrays into Pandas DataFrames so they are easier to explore and work with
  X_train = pd.DataFrame(data=X_train, index=train_index)
  y_train = pd.Series(data=y_train, index=train_index)
  X_test = pd.DataFrame(data=X_test, index=test_index)
  y_test = pd.Series(data=y_test, index=test_index)
 
eval_set = [(X_train, np.ravel(y_train)), (X_test, np.ravel(y_test))]
 
# Set the hyperparameters
 
base_score = 0.5
colsample_bylevel = 1
colsample_bytree = 1
gamma = 0
learning_rate = 0.1
max_depth = 3
min_child_weight = 1
n_estimators = 100
subsample =1
n_jobs = 1
colsample_bytree = 0.5
gamma = 0
reg_lambda= 1
scale_pos_weight= 1
random_state = 42
 
 
xgb_clf1 = XGBClassifier(learning_rate= learning_rate,
                         base_score = base_score,
                         max_depth= max_depth,
                         n_jobs = n_jobs,
                         min_child_weight= min_child_weight,
                         n_estimators= n_estimators,
                         subsample =subsample,
                         colsample_bytree = colsample_bytree,
                         gamma = gamma,
                         scale_pos_weight = scale_pos_weight,
                         reg_lambda = reg_lambda,
                         random_state = random_state)
 
 
xgb_clf1.fit(X_train, y_train, 
             eval_metric = 'auc', 
             eval_set = eval_set,
             early_stopping_rounds = 10, 
             verbose = 10)
 
ntree_limit = xgb_clf1.best_ntree_limit
pred_prob = xgb_clf1.predict_proba(X_test, ntree_limit = ntree_limit)[:, 1]
AUROC = roc_auc_score(y_test, pred_prob)
print('Area under ROC curve: %.2f%%' %  (AUROC*100))
print('\n')
 
# make predictions for test data
y_pred = xgb_clf1.predict(X_test, ntree_limit = ntree_limit)
 
results = confusion_matrix(y_test, y_pred) 
 
print('Confusion Matrix :')
print(results)
print('\n') 
 
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print('\n')
 
print ('Report : ')
print (classification_report(y_test, y_pred)) 
 
# Area under Curve
pred_prob = xgb_clf1.predict_proba(X_test, ntree_limit = ntree_limit)[:, 1]
AUROC = roc_auc_score(y_test, pred_prob)
print('Area under ROC: %.2f%%' %  (AUROC*100))
print('\n')
 
#print('\033[4mNext day Open price direction\033[0m')
pred = pd.DataFrame(y_pred)
pred = (pred[:5])
 
# assigning date to the predicted values
df['price_direction'] = np.nan
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day 
 
for i in pred:
  next_date = datetime.datetime.fromtimestamp(next_unix)
  next_unix += one_day
  df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
 
#create an index of just the date portion of our index (this is the slow step)
ts_days = pd.to_datetime(df['price_direction'].index.date)
 
#create a range of business days over that period
bdays = pd.bdate_range(start=df['price_direction'].index[0].date(), end=df['price_direction'].index[-1].date(), freq = 'B')
 
# assigning dates to dataframes
a = df.sort_values(by='Date', ascending=False)
pred.index = a.index[:5]
print(pred.index)
 
# assigning names to columns
pred.rename(columns = {0: 'price_direction'}, inplace=True)
pred
 
 
# For each day where open price > 0, we set the signal 1 means sell  and < 0, we set signal 0, to buy 
print('\033[4mFor each day where open price = 0 = black arrow head  and 1 = green arrow head \033[0m')
buys = df.loc[df['label'] == 0]
sells = df.loc[df['label'] == 1]
 
# Plot 
fig = plt.figure(figsize=(20, 5))
plt.plot(df.index, df['Open'], color = 'b', lw=2., label='Open')
 
# Plot the buy and sell signals on the same plot
plt.plot(buys.index, df.loc[buys.index]['Open'], '^', markersize=5, color='k',
         lw=2., label='Buy')
plt.plot(sells.index, df.loc[sells.index]['Open'], 'v', markersize=5, color='g',
         lw=2., label='Sell')
plt.ylabel('Price')
plt.xlabel('Date')
plt.title('Buy and sell signals plot')
plt.legend(loc=0)
 
# Display everything
plt.show()
print('\n')
 
ypred = (y_pred > 0.5)
df['ypred'] = np.NaN
df.iloc[(len(df) - len(ypred)):,-1:] = ypred
trade_dataset = df.copy()
td = trade_dataset['ypred'].tail()
forecast = pd.concat([td, pred], axis=1) # combining dataframes to see actutal vs predicted
forecast = forecast.reset_index()
forecast.sort_values(by=['Date'], inplace=True, ascending=False)
# dropping the weekends
forecast = forecast[forecast['Date'].apply(lambda x: x.weekday() not in [5,6])]
print(forecast)
 
## Moving averages crossover 
df1 = df.apply(copy.deepcopy)
 
# moving average crossover :: sma 100 is > sma 10
df1['target'] = np.where(df['sma2'] > df1['sma5'], 1,0)
#df1 = df1.fillna(0)
df1['target'].tail()
#print('\n')
 
def getBinary(val):
    if val<0:
        return 0
    else:
        return 1
df1['next_day_direction'] = df1["target"].apply(getBinary)
 
# Plot 
fig = plt.figure(figsize=(20,6))
plt.plot(df1['Last'], color='gray', label='Last')
# Plot the buy and sell signals on the same plot
#  When the shorter-term MA crosses above the longer-term MA (Golden cross), it's a buy signal, it indicates that the trend is shifting up. 
#  When the shorter-term MA crosses below the longer-term MA, it's a sell signal (dead/death cross), it indicates that the trend is shifting down. 
plt.plot(df1['sma3'].dropna(), color='r', label = 'sma20')
plt.plot(df1['sma5'].dropna(), color='g', label = 'sma100')
plt.ylabel('Price')
plt.xlabel('Date')
plt.suptitle('SMA crossover')
plt.title('Buy -> Shorter MA crosses below longer MA (Deadcross); Sell -> Shorter MA crosses above longer MA (Golden cross)')
plt.legend(loc=0)
 
# Display everything
plt.show()
print('\n')
 
 
## 5 days look-ahead Open price
 
df4 = merge12.apply(copy.deepcopy)
 
sma1 = 3
sma2 = 10
sma3 = 20
sma4 = 30
sma5 = 100
df4["sma1"] = df4['Open'].shift(1).rolling(sma1).mean()
df4["sma2"] = df4['Open'].shift(1).rolling(sma2).mean()
df4['sma3'] = df4['Open'].shift(1).rolling(sma3).mean()
df4['sma4'] = df4['Open'].shift(1).rolling(sma4).mean()
df4['sma5'] = df4['Open'].shift(1).rolling(sma5).mean()
df4['Std_dev']= df4['Open'].rolling(5).std()
 
df4['ema_12'] = df4['Open'].ewm(span=10).mean()
df4['ema_26'] = df4['Open'].ewm(span=26).mean()
df4['ROC'] = ((df4['Open'] - df4['Open'].shift(5)) / (df4['Open'].shift(5)))*100
 
delta = df4['Open'].diff()
window = 14
up_days = delta.copy()
up_days[delta<=0]=0.0
down_days = abs(delta.copy())
down_days[delta>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
df4['rsi'] = 100-100/(1+RS_up/RS_down)
df4['macd'] = df4['ema_12'] - df4['ema_26']
df4['Williams%R'] = (df4['High'].max() - df4['Last'])/(df4['High'] - df4['Low'].min()) * -100
 
# WINDOW FEATURE
df4['t-2'] = df4['Open'].shift(2)
df4['t-1'] = df4['Open'].shift(1)
 
# ROLLING WINDOW FEATURE
width = 3
shifted = df4['Open'].shift(width - 1)
roll_window = shifted.rolling(window=width)
df4['min'] = roll_window.min()
df4['mean'] = roll_window.mean()
df4['max'] = roll_window.max()
 
# EXPANDING WINDOW STATISTICS
exp_window = df4['Open'].expanding()
df4['min_exp'] = exp_window.min()
df4['mean_exp'] = exp_window.mean()
df4['max_exp'] = exp_window.max()
 
df4 = df4.astype(float)
 
# A variable for predicting 'n' days out into the future
n = int(5) #'n=5' days
future_pred = n 
 
#Create another column (the target or dependent variable) shifted 'n' units up
df4['Prediction'] = df4['Open'].shift(-future_pred)
 
X3 = np.array(df4.drop(columns = ['Prediction',
                                  'day_of_week',
                                  'Change',
                                  'Volume',
                                  'Open',
                                  'High',
                                  'Low',
                                  'Last',
                                  'Settle',
                                  'Previous Day Open Interest'], axis=1))
                         
X3_forecast = X3[-future_pred:] # set X3_forecast equal to last 5, we do not have y3 values for X3_forecast
X3 = X3[:-future_pred] # remove last 5 from X3
 
y3 = np.array(df4['Prediction'])
y3 = y3[:-future_pred]
 
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.33, random_state=42)
 
tscv = TimeSeriesSplit()
#print(tscv)
TimeSeriesSplit(max_train_size=0.67, n_splits=5)
for train_index, test_index in tscv.split(X3):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X3_train, X3_test = X3[train_index], X3[test_index]
  y3_train, y3_test = y3[train_index], y3[test_index]
 
eval_set = [(X3_train, np.ravel(y3_train)), (X3_test, np.ravel(y3_test))]
 
from xgboost import XGBRegressor
import xgboost as xgb
 
reg2 = xgb.XGBRegressor(objective ='reg:squarederror',
                        colsample_bytree = 0.3, 
                        learning_rate = 0.1, 
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 100,
                        random_state = 42)
 
 
reg2.fit(X3_train, y3_train, 
         eval_metric = 'rmse', 
         eval_set = eval_set,
         early_stopping_rounds = 10, 
         verbose = 10)
 
open_confidence = reg2.score(X3_test, y3_test)
print("open_confidence: ", round(open_confidence*100,2).astype(str) + '%')
print('\n')
 
forecast_pred_2 = reg2.predict(X3_forecast)
rmse1 = np.sqrt(mean_squared_error(y3_test[:future_pred], forecast_pred_2[:future_pred]))
 
xgb_open_pred = reg2.predict(X3_test) # prediction on test data
xgb_open_pred = np.sort(xgb_open_pred) # sorting data in decending order
xgb_open_pred = pd.DataFrame(xgb_open_pred)
xgb_open_pred = round(xgb_open_pred[:5],2)
xgb_open_pred.rename(columns = {0: 'prediction'}, inplace=True)
d = df4.sort_values(by='Date', ascending=False)
xgb_open_pred.index = d.index[:5]
combine_open = pd.concat([df4['Open'].tail(), xgb_open_pred], axis=1) # combining dataframes to see actutal vs predicted
combine_open['accuracy'] = round(combine_open.apply(lambda row: row.prediction / 
                                            row.Open *100, axis = 1),2).astype(str) + '%' 
combine_open = combine_open.reset_index()
 
 
fig = go.Figure()
 
fig.add_trace(go.Scatter(
    x=combine_open['Date'],
    y=combine_open['Open'],
    name="Actual open price"       # this sets its legend entry
))
 
 
fig.add_trace(go.Scatter(
    x=combine_open['Date'],
    y=combine_open['prediction'],
    name="Predicted open price"
))
 
fig.update_layout(
    title="Actual vs Predicted Open Price",
    xaxis_title="Date",
    yaxis_title="Price ($/Mbtu)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
 
fig.show()
 
 
df4['open_pred'] = np.nan
last_date = df4.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day 
 
for i in forecast_pred_2:
  next_date = datetime.datetime.fromtimestamp(next_unix)
  next_unix += one_day
  df4.loc[next_date] = [np.nan for _ in range(len(df4.columns)-1)] + [i]
 
ts_days = pd.to_datetime(df4['open_pred'].index.date)
 
bdays = pd.bdate_range(start=df4['open_pred'].index[0].date(), end=df4['open_pred'].index[-1].date())
df4['open_pred'] = round(df4.open_pred[ts_days.isin(bdays)],2)
 
 
## Close price
df3 = merge12.apply(copy.deepcopy)
 
sma1 = 3
sma2 = 10
sma3 = 20
sma4 = 30
sma5 = 100
df3["sma1"] = df3['Last'].shift(1).rolling(sma1).mean()
df3["sma2"] = df3['Last'].shift(1).rolling(sma2).mean()
df3['sma3'] = df3['Last'].shift(1).rolling(sma3).mean()
df3['sma4'] = df3['Last'].shift(1).rolling(sma4).mean()
df3['sma5'] = df3['Last'].shift(1).rolling(sma5).mean()
df3['Std_dev']= df3['Last'].rolling(5).std()
 
df3['ema_12'] = df3['Last'].ewm(span=10).mean()
df3['ema_26'] = df3['Last'].ewm(span=26).mean()
df3['ROC'] = ((df3['Last'] - df3['Last'].shift(5)) / (df3['Last'].shift(5)))*100
 
delta = df3['Last'].diff()
window = 14
up_days = delta.copy()
up_days[delta<=0]=0.0
down_days = abs(delta.copy())
down_days[delta>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
df3['rsi'] = 100-100/(1+RS_up/RS_down)
df3['macd'] = df3['ema_12'] - df3['ema_26']
df3['Williams%R'] = (df3['High'].max() - df3['Last'])/(df['High'] - df3['Low'].min()) * -100
 
df3['t-2'] = df3['Last'].shift(2)
df3['t-1'] = df3['Last'].shift(1)
width = 3
shifted = df3['Last'].shift(width - 1)
roll_window = shifted.rolling(window=width)
df3['min'] = roll_window.min()
df3['mean'] = roll_window.mean()
df3['max'] = roll_window.max()
exp_window = df3['Last'].expanding()
df3['min_exp'] = exp_window.min()
df3['mean_exp'] = exp_window.mean()
df3['max_exp'] = exp_window.max()
 
df3 = df3.astype(float)
 
n = int(5) 
pred_future = n 
 
df3['prediction'] = df3['Last'].shift(-pred_future)
 
X2 = np.array(df3.drop(columns = ['prediction',
                                  'day_of_week',
                                  'Change',
                                  'Open',
                                  'High',
                                  'Low',
                                  'Last',
                                  'Settle',
                                  'Volume',
                                  'Previous Day Open Interest'], axis=1))
                         
X2_forecast = X2[-pred_future:] 
X2 = X2[:-pred_future] 
y2 = np.array(df3['prediction'])
y2 = y2[:-pred_future]
 
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.33, random_state=42)
 
tscv = TimeSeriesSplit()
#print(tscv)
TimeSeriesSplit(max_train_size=0.67, n_splits=5)
for train_index, test_index in tscv.split(X2):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X2_train, X2_test = X2[train_index], X2[test_index]
  y2_train, y2_test = y2[train_index], y2[test_index]
 
eval_set = [(X2_train, np.ravel(y2_train)), (X2_test, np.ravel(y2_test))]
 
reg1 = xgb.XGBRegressor(objective ='reg:squarederror',
                        colsample_bytree = 0.3, 
                        learning_rate = 0.1, 
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 100,
                        random_state = 42)
 
 
reg1.fit(X2_train, y2_train, 
         eval_metric = 'rmse', 
         eval_set = eval_set,
         early_stopping_rounds = 10, 
         verbose = 10)
 
print('\033[4mModel performance :: Regression Quality\033[0m')
plt.rcParams['figure.figsize'] = 15, 5 
plt.rcParams['font.size'] = 12
 
close_confidence = reg1.score(X2_test, y2_test)
print("close_confidence: ", round(close_confidence*100,2).astype(str) + '%')
print('\n')
 
forecast_pred_1 = reg1.predict(X2_forecast)
rmse2 = np.sqrt(mean_squared_error(y2_test[:pred_future], forecast_pred_1[:pred_future]))
print('Close Test RMSE: %.2f' % rmse2)
print('\n')
 
xgb_close_pred = reg1.predict(X2_test) # prediction on test data
xgb_close_pred = np.sort(xgb_close_pred) # sorting data in decending order
xgb_close_pred = pd.DataFrame(xgb_close_pred)
xgb_close_pred = round(xgb_close_pred[:5],2)
xgb_close_pred.rename(columns = {0: 'prediction'}, inplace=True)
e = df3.sort_values(by='Date', ascending=False)
xgb_close_pred.index = e.index[:5]
combine_close = pd.concat([df3['Last'].tail(), xgb_close_pred], axis=1) # combining dataframes to see actutal vs predicted
combine_close['accuracy'] = round(combine_close.apply(lambda row: row.prediction / 
                                          row.Last *100, axis = 1),2).astype(str) + '%'
combine_close = combine_close.reset_index()
 
# Plot actural open vs prediction
fig = go.Figure()
 
fig.add_trace(go.Scatter(
    x=combine_close['Date'],
    y=combine_close['Last'],
    name="Actual Close price"       # this sets its legend entry
))
 
 
fig.add_trace(go.Scatter(
    x=combine_close['Date'],
    y=combine_close['prediction'],
    name="Predicted Close price"
))
 
fig.update_layout(
    title="Actual vs Predicted Close Price",
    xaxis_title="Date",
    yaxis_title="Price",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
 
fig.show()
 
df3['close_pred'] = np.nan
last_date = df3.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day 
 
for i in forecast_pred_1:
  next_date = datetime.datetime.fromtimestamp(next_unix)
  next_unix += one_day
  df3.loc[next_date] = [np.nan for _ in range(len(df3.columns)-1)] + [i]
 
ts_days = pd.to_datetime(df3['close_pred'].index.date)
bdays = pd.bdate_range(start=df3['close_pred'].index[0].date(), end=df3['close_pred'].index[-1].date())
df3['close_pred'] = round(df3.close_pred[ts_days.isin(bdays)],2)
 
h_stack = pd.concat([df4['open_pred'], df3['close_pred']], axis=1)
h_stack = h_stack.reset_index()
h_stack.tail()
 
 
NG1 = quandl.get("CHRIS/CME_NG1", authtoken="LSQpgUzwJRoF667ZpzyL") # natural gas continuous contract 1
NG1 = NG1.loc['2000-01-01':,['Open', 'High', 'Low', 'Last', 'Volume']]
NG1.reset_index(level=0, inplace=True)
 
NG1['week_no'] = NG1['Date'].dt.week
NG1['month_no'] = NG1['Date'].dt.month
NG1['year'] = NG1['Date'].dt.year
NG1['DayofWeek'] = NG1['Date'].dt.dayofweek
 
monthly = NG1.pivot_table(values= 'Last', columns = ['year'],aggfunc = np.mean, index = ['month_no'])
 
monthly.loc[0] = monthly.loc[12,:].shift(1)
monthly.fillna(method='ffill', inplace=True) # filling nan values with previous data
monthly = monthly.sort_index()
print(round(monthly,2))
 
monthly_change = monthly.pct_change()
monthly_change = monthly_change.drop(monthly_change.index[0])
 
monthly_change['raise'] = monthly_change[(monthly_change.iloc
                                                              [:,:]>0)].count(axis=1)/(2020-2000)
monthly_change['decline'] = monthly_change[(monthly_change.iloc
                                                              [:,:]<0)].count(axis=1)/(2020-2000)   
monthly_change = monthly_change.reset_index()
 
 
#####-------Dashboard--Program---------######
 
NG = NG.reset_index()
combine_open['prediction'] = combine_open['prediction'].map('{:,.2f}'.format)
combine_close['prediction'] = combine_close['prediction'].map('{:,.2f}'.format)
 
colors = {
    'background': '#111222',
    'text': '#7FDBFF'
}
 
 
app.layout=html.Div(
    [
     html.Div(
            children='Natural Gas Contract',
            style={
                    'textAlign': 'center',
                    'color': colors['text'],
                    'fontSize':45,
                    'background': '#111222'
                  }
      
            ),
     dcc.Tabs(
         [
          dcc.Tab(
              label='Spot Price',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize': 25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },
              children=[
                html.Div(
                  [
                    dcc.Graph(id='spot price',
                      figure={'data':[
                                      {'x':hh_spot.index,'y':hh_spot['hh_spot'],'type':'line','name':'Spot Price'},
                                      ],
                              'layout':{
                                      #'title':'Spot Price ->' + str(hh_spot.hh_spot.tail(1)),
                                      'annotations':[{
                                          'text':'Spot Price: ' + ((str(hh_spot['hh_spot'].tail(1)).split('Name')[0]).split()[2]) + ' Dollars/ MBtu',
                                          'fontSize':20,
                                          'color':colors['text'],
                                          'showarrow':False,
                                          'align':'right',
                                          'x':1,
                                          'y':1,
                                          'xref':'paper',
                                          'yref':'paper'
                                      },
                                      {
                                          'text':'Week: ' + ((str(hh_spot['hh_spot'].tail(1)).split('Name')[0]).split()[1]),
                                          'color':colors['text'],
                                          'showarrow':False,
                                          'align':'right',
                                          'x':1,
                                          'y':0.9,
                                          'xref':'paper',
                                          'yref':'paper'
                                      }],
                                      'plot_bgcolor':colors['background'],
                                      'paper_bgcolor':colors['background'],
                                      'font':{
                                          'color':colors['text']
                                              },
                                        'xaxis':{
                                            'title':'Date',
                                            'rangeslider': {'visible':True},},
                                        'yaxis':{'title':'Price ($ / Mbtu)'}
                                        #'marginBottom':'10px'
                                      }},
                              config={
                                  'displayModeBar': True,
                                  'displaylogo': False,                                       
                                  'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                                },)
                
                ],
                style={'width': '100%', 'display': 'inline-block'},
                ),
 
                
              ]),
          
          dcc.Tab(
              label='Natural Gas',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize': 25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },
              children=[
                html.Div(
                  [
                    dcc.Graph(id='NaturalGas',
                      figure={'data':[{
                                'x':NG['Date'],
                                'open':NG['Open'],
                                'high':NG['High'],
                                'low':NG['Low'],
                                'close':NG['Last'],
                                'type':'candlestick'
                                    }],
                              'layout':{
                                      'title':"Natural Gas Futures Prices",
                                      'plot_bgcolor':colors['background'],
                                      'paper_bgcolor':colors['background'],
                                      'font':{
                                          'color':colors['text']
                                              },
                                        'yaxis':{'title':'Contract #1 ($/Mbtu)'}
                                        #'marginBottom':'10px'
                                      }},
                              config={
                                  'displayModeBar': True,
                                  'displaylogo': False,                                       
                                  'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                                },)
                
                ],
                style={'width': '100%', 'display': 'inline-block'},
                ),
 
                html.Div(
                [
                  html.Div(
                    children='  LATEST VALUES',
                    style={
                        'textAlign': 'center',
                        'color': colors['text'],
                        'fontSize':20,
                        'height':35,
                        'background': '#111222',
                        #'marginLeft':'10px'
                        }
            
                        ),
                 
                   html.Div(
                    [
                         dbc.Card(
                          [html.Div("Date",className="card-name",style={'marginTop':'0%','marginLeft':'25%'}),
                          html.P(
                             (str(NG['Date'].tail(1)).split('Name')[0]).split()[1],
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 'marginLeft':'25%'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                                      
                  
                  html.Div(
                    [
                         dbc.Card(
                         [html.Div("Open Price",className="card-name",style={'marginTop':'0%'}),
                          html.P(
                             round(NG['Open'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                 
                  html.Div(
                    [
                         dbc.Card(
                          [html.Div("Close Price",className="card-name",style={'marginTop':'0%'}),
 
                          html.P(
                             round(NG['Last'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                 
                 html.Div(
                    [
                         dbc.Card(
                          [html.Div("Adj Close Price",className="card-name",style={'marginTop':'0%'}),
 
                          html.P(
                            round(NG['Settle'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                                    
                    
                ]
                ),
                html.Div(
                 [
                 
                 html.Div(
                    [
                         dbc.Card(
                         [html.Div("High Price",className="card-name",style={'marginTop':'0%','marginLeft':'50%','background': '#111222'}),
                          html.P(
                             round(NG['High'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 'marginLeft':'50%'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '50%', 'display': 'inline-block','marginBottom':'0.5%','height':'40%'},
                    ),
                  html.Div(
                    [
                         dbc.Card(
                         [html.Div("Low Price",className="card-name",style={'marginTop':'0%'}),
                          html.P(
                             round(NG['Low'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 #'marginLeft':'440px'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','marginBottom':'0.5%','height':'40%'},
                    ),
                 html.Div(
                    [
                         dbc.Card(
                         [html.Div("Volume",className="card-name",style={'marginTop':'0%'}),
                          html.P(
                             NG['Volume'].tail(1),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 #'marginLeft':'70px'
                             }
                          ),
                           
                           ],
 
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','marginBottom':'0.5%','height':'40%'},
                    ),
                 ]),
                  
                  html.Div(
                     [
                      html.Div(
                        children='Last 5 observed prices',
                        style={
                          'textAlign': 'center',
                          'color': colors['text'],
                          'fontSize':20,
                          'background': '#111222'
                          }
                        ),
         
                      html.Div(
                          [
                          dash_table.DataTable(
                            id='last_openclose',
                            columns=[{"name": 'Date', "id": 'Date','type':'datetime'},
                                     {"name": 'Open', "id": 'Open'},
                                     {"name": 'Close', "id": 'Last'},
                                     {"name": 'High', "id": 'High'},
                                     {"name": 'Low', "id": 'Low'},
                            
                                     ],
                            data= ng.tail().to_dict('rows'),
                            style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                          'fontWeight':'bold',
                                          'color': colors['text'],
                                          'textAlign':'center'
                                          },
                            style_cell={
                                'backgroundColor': 'rgb(50, 50, 50)',
                                'color': colors['text'],
                                'textAlign':'center'
                                },
                            )
                          ],
 
                        )
                      ]
                      ),
 
                       #cards 0  
                        html.Div(
                        [
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg Sharpe daily %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(sharpe_ratio,2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '16%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Annualized Sharpe %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(((252**0.5) * sharpe_ratio),2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '17%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg. Daily returns %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round((mean_return*100),2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '16%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg. Volatility %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round((return_stdev*100), 2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '13%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg ann return %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [ round(((mean_return * 252)*100),2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Ann. Volatility %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(((return_stdev * np.sqrt(252))*100), 2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '15%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            )
                        
                            
                            
                        ]
                        ),
                    #cards 1 
                    html.Div(
                    [
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Min daily %\n return",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(daily_ret.min(),2)*100) +' on '+str(daily_ret.idxmin())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                                          
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Max daily %\n return",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(daily_ret.max()*100, 2)) +' on '+str(daily_ret.idxmax())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Min. Price\n    (2000 - till date)",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(df['Last'].min(),2))+' on '+str(df['Last'].idxmin())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Max. Price\n    (2000 - till date)",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(df['Last'].max(),2))+' on '+ str(df['Last'].idxmax())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                        
                        
                    ]
                    ),
                    #cards 2 
                        html.Div(
                        [
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability +/-(1)% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.01) & (daily_ret < 0.01)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability +/-(3)% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.03) & (daily_ret < 0.03)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability +/-(5)% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.05) & (daily_ret < 0.05)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability > 5% price change ",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[daily_ret > 0.05].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability < 5% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[daily_ret < -0.05].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                            
                            
                        ]
                        ),
              ]),
             
          
          dcc.Tab(
              label='Buy/Sell Direction',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                    dcc.Graph(id='BuysSells',
                        figure={'data':[
                          {'x':df.index,'y':df['Last'],'type':'line','color':'blue','name':'Last'},
                          {'x':buys.index,'y':df.loc[buys.index]['Last'],'mode':'markers','name':'Buys','marker':{'color':'red'}},
                          {'x':sells.index,'y':df.loc[sells.index]['Last'],'mode':'markers','name':'Sells','marker':{'color':'green'}},
                          
                      ],
                      'layout':{
                        'title':'Buy (Red) and Sell (Green) Signals',
                        'plot_bgcolor':colors['background'],
                        'paper_bgcolor':colors['background'],
                        'font':{
                            'color':colors['text']
                        },
                        'rang':[-0.6,0.6],
                        'marginBottom':'10px',
                        'paddingTop':'10px',
                        'xaxis':{
                            'title':'Date'
                        },
                        'yaxis':{
                            'title':'Price'
                        }
                              }
                        },
                        config={
                            'displayModeBar': True,
                            'displaylogo': False,                                       
                            'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                          },
                      ),
 
                      
 
                      html.Div(
                        [
                            html.Div(
                                [html.Div(
                                  children='Open Price Prediction',
                                  style={
                                    'textAlign': 'center',
                                    'color': colors['text'],
                                    'fontSize':25,
                                    'background': '#111222',
                                    #'display':'flex',
                                    'width':'100%',
                                    'height':'19%',
                                    'paddingTop':'3.5%'
                                    }
                                  ),
                            dash_table.DataTable(
                              id='open_pred_table',
                              columns=(
                              [{"name": 'Date', "id": 'Date','type':'datetime'},
                               {"name": 'Prediction', "id": 'prediction'},
                               {"name": 'Open', "id": 'Open'},
                               {"name": 'Accuracy', "id": 'accuracy'},
                               ]
                      
                              ),
                              data=(combine_open.to_dict('rows')),
                              editable =True,
                              style_table={
                                  #'maxHeight': '50ex',
                                  #'overflowY': 'scroll',
                                  #'display':'flex',
                                  'width': 633,
                                  'minWidth': 250,
                              },
                              style_cell={
                                  'backgroundColor': 'rgb(50, 50, 50)',
                                  'color': colors['text'],
                                  'textAlign':'center'
                              },
                              style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                            'fontWeight':'bold',
                                            'color': colors['text'],
                                            'textAlign':'center'},
                              
                              
                              ),
                              ]),
 
                            dcc.Graph(id='open_predicition',
                        figure={'data':[
                          {'x':combine_open['Date'],'y':combine_open['Open'],'type':'line','name':'Actual'},
                          {'x':combine_open['Date'],'y':combine_open['prediction'],'type':'line','name':'Predicted'},
                      ],
                      'layout':{
                        'title':'Last 5 Actual vs Predicted Open Price',
                        'plot_bgcolor':colors['background'],
                        'paper_bgcolor':colors['background'],
                        'font':{
                            'color':colors['text']
                        },
                        'rang':[-0.6,0.6],
                        'marginBottom':'10px',
                        'paddingTop':'10px',
                        'xaxis':{
                            'title':'Date'
                        },
                        'yaxis':{
                            'title':'Price ($ / Mbtu)'
                        },
                        'autoSize':False,
                        'width':700,
                        'height':250
                              }
                              
                        },
                        config={
                            'displayModeBar': False,
                            'displaylogo': False,                                       
                            'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                          },
                      ),
 
                        ],
                        style={'width':'100%','display':'flex','marginTop':'0.5%'}
                      ),
 
                      html.Div(
                        [
                            html.Div(
                                [html.Div(
                                  children='Close Price Prediction',
                                  style={
                                    'textAlign': 'center',
                                    'color': colors['text'],
                                    'fontSize':25,
                                    'background': '#111222',
                                    #'display':'flex',
                                    'width':'100%',
                                    'height':'19%',
                                    'paddingTop':'3.5%'
                                    }
                                  ),
                            dash_table.DataTable(
                              id='close_pred_table',
                              columns=(
                              [{"name": 'Date', "id": 'Date','type':'datetime'},
                               {"name": 'Prediction', "id": 'prediction'},
                               {"name": 'Close', "id": 'Last'},
                               {"name": 'Accuracy', "id": 'accuracy'},
                               ]
                      
                              ),
                              data=(combine_close.to_dict('rows')),
                              editable =True,
                              style_table={
                                  #'maxHeight': '50ex',
                                  #'overflowY': 'scroll',
                                  #'display':'flex',
                                  'width': 633,
                                  'minWidth': 250,
                              },
                              style_cell={
                                  'backgroundColor': 'rgb(50, 50, 50)',
                                  'color': colors['text'],
                                  'textAlign':'center'
                              },
                              style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                            'fontWeight':'bold',
                                            'color': colors['text'],
                                            'textAlign':'center'},
                              
                              
                              ),
                              ]),
 
                            dcc.Graph(id='close_predicition',
                        figure={'data':[
                          {'x':combine_close['Date'],'y':combine_close['Last'],'type':'line','name':'Actual'},
                          {'x':combine_close['Date'],'y':combine_close['prediction'],'type':'line','name':'Predicted'},
                      ],
                      'layout':{
                        'title':'Last 5 Actual vs Predicted Close Price',
                        'plot_bgcolor':colors['background'],
                        'paper_bgcolor':colors['background'],
                        'font':{
                            'color':colors['text']
                        },
                        'rang':[-0.6,0.6],
                        'marginBottom':'10px',
                        'paddingTop':'10px',
                        'xaxis':{
                            'title':'Date'
                        },
                        'yaxis':{
                            'title':'Price ($ / Mbtu)'
                        },
                        'autoSize':False,
                        'width':700,
                        'height':250
                              }
                              
                        },
                        config={
                            'displayModeBar': False,
                            'displaylogo': False,                                       
                            'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                          },
                      ),
 
                        ],
                        style={'width':'100%','display':'flex','marginTop':'0.5%'}
                      ),
 
                      
                      
                      html.Div(
                          [
                           html.Div(
                              [
                                html.Div(
                                  children='Next 5 Days Open/Close price',
                                  style={
                                    'textAlign': 'center',
                                    'color': colors['text'],
                                    'fontSize':20,
                                    'background': '#111222'
                                    }
                                  ),
                                
                                html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Open Confidence: "+str(round(open_confidence*100,2))+"%"],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft': '58%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                          
                                        ],
                                        style={'width': '70%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(["Close Confidence: "+str(round(close_confidence*100,2))+"%"],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              #'marginRight':'40%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                        ],
                                        style={'width': '30%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    )
                                        
                                ]),
                               
                               html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Open RMS Error: "+str(round(rmse1,2))],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft': '58%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              #'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                          
                                        ],
                                        style={'width': '70%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(["Close RMS Error: "+str(round(rmse2,2))],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              #'marginRight':'40%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              #'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                        ],
                                        style={'width': '30%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    )
                                        
                                ]),
                              
                                
                  
                                html.Div(
                                    [
                                    dash_table.DataTable(
                                      id='next_openclose',
                                      columns=[{"name": 'Date', "id": 'Date','type':'datetime'},
                                              {"name":'Open Prediction','id':'open_pred'},
                                              {"name":'Close Prediction','id':'close_pred'}],
                                      data= h_stack.tail().to_dict('rows'),
                                      style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                    'fontWeight':'bold',
                                                    'color': colors['text'],
                                                    'textAlign':'center'
                                                    },
                                      style_cell={
                                          'backgroundColor': 'rgb(50, 50, 50)',
                                          'color': colors['text'],
                                          'textAlign':'center'
                                          },
                                      )
                                    ],
 
                                  )
                                ],
                                style={'width': '50%', 'display': 'inline-block','marginTop':'0.5%'},
                                ),
 
                                html.Div(
                                  [
                                    html.Div(
                                        children='Open price direction',
                                        style={
                                            'textAlign': 'center',
                                            'color': colors['text'],
                                            'fontSize':22,
                                            'background': '#111222',
                                            'height':30,
                                            }
                                          ),
                                   html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Accuracy: %.2f%%" % (accuracy * 100.0)],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft': '0%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':18,
                                              #'fontWeight':'bold',
                                              'background': '#111222',
                                              'height':30
                                              })
                                          
                                        ],
                                        style={'width': '50%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(['Area under curve: %.2f%%' %  (AUROC*100)],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft':'0%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':18,
                                              #'fontWeight':'bold',
                                              'background': '#111222',
                                              'height':30
                                              })
                                        ],
                                        style={'width': '50%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    )
                                        
                                ]),
                                    html.Div(
                                    [
                                        dash_table.DataTable(
                                            id='open_direction',
                                            columns =(
                                                [
                                                  {'name':'Date', 'id':'Date','type':'datetime'},
                                                 {'name':'Price Direction','id':'price_direction'},
                                                 {'name':'Trading Strategy','id':'ypred'}
                                                ]
                                            ),
                                            
                                            data=(forecast.to_dict('records')),
                                            style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                          'fontWeight':'bold',
                                                          'color': colors['text'],
                                                          'textAlign':'center'},
                                            style_cell={
                                                'backgroundColor': 'rgb(50, 50, 50)',
                                                'color': colors['text'],
                                                'textAlign':'center'
                                            },
                                            
                                          
                                        )
                                    ],
                                    )
                                  ],
                                  style={'width': '48%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'35%'},
                                  ),
                          ]
                      )
                      ]
                  ),
          
           dcc.Tab(
              label='SMA Crossover',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                        html.Div(
                          [
                            dcc.Graph(id='SMACrossover',
                            figure={'data':
                              [
                                {'x':df1.index,'y':df1['Last'],'type':'line','name':'Close'},
                                {'x':df1.index,'y':df1['sma3'],'type':'line','name':'SMA20'},
                                {'x':df1.index,'y':df1['sma5'],'type':'line','name':'SMA100'},
                              ],
                              'layout':{
                                  'title':'Simple Moving Average Crossover',
                                  'annotations':[{
                                      'text':'Sell -> Shorter MA crosses above longer MA (Golden cross)',
                                      'color':colors['text'],
                                      'showarrow':False,
                                      'align':'right',
                                      'x':1,
                                      'y':1,
                                      'xref':'paper',
                                      'yref':'paper'
                                  },
                                  {
                                      'text':'Buy -> Shorter MA crosses below longer MA (Dead Cross)',
                                      'color':colors['text'],
                                      'showarrow':False,
                                      'align':'right',
                                      'x':1,
                                      'y':0.9,
                                      'xref':'paper',
                                      'yref':'paper'
                                  }],
                                  'plot_bgcolor':colors['background'],
                                  'paper_bgcolor':colors['background'],
                                  'font':{
                                      'color':colors['text']
                                  },
                                  'rang':[-0.6,0.6],
                                  'marginBottom':'10%',
                                  'xaxis':{
                                      'title':'Date'
                                  },
                                  'yaxis':{
                                      'title':'Price'
                                  }
                              }
                              },
                              config={
                                'displayModeBar': True,
                                'displaylogo': False,                                       
                                'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                              },)                        
                                              
                                              
                            ]),
                        
                       
                        ],
              ),
          
           dcc.Tab(
              label='Drawdown',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                        html.Div(
                          [
                            dcc.Graph(id='DailyDrawdown',
                                      figure={'data':[{
                                            'x':daily_drawdown.index,
                                            'y':daily_drawdown,
                                            'type':'line'
                                        },
                                        
                                        {
                                            'x':min_daily_drawdown.index,
                                            'y':min_daily_drawdown,
                                            'type':'line'
                                        }
                                    ],'layout':{
                                            'title':'Maximum Daily Drawdown',
                                            'plot_bgcolor':colors['background'],
                                            'paper_bgcolor':colors['background'],
                                            'font':{
                                                'color':colors['text']
                                            },
                                            'marginBottom':'10px',
                                            'xaxis':{
                                                'title':'Date'
                                            }            
                                        }},
                                      config={
                                        'displayModeBar': True,
                                        'displaylogo': False,                                       
                                        'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                                      },
                                      )                        
                          ],
                          style={'width': '100%', 'display': 'inline-block'},
                        ),
 
                        
                        html.Div(
                          [
                            html.Div(
                              children='Monthly Gas Price Decline/Raise',
                              style={
                                  'textAlign': 'center',
                                  'color': colors['text'],
                                  'fontSize':20,
                                  'background': '#111222'
                                  }
                              ),
                            html.Div(
                            [
                                dash_table.DataTable(
                                    id='raise_decline',
                                    columns=(
                                    [{"name": 'Month', "id": 'month_no'},
                                    {"name": 'Raise', "id": 'raise'},
                                    {"name": 'Decline', "id": 'decline'},
                                    ]
                                    ),
                                    data=(monthly_change.to_dict('records')),
                                    style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                  'fontWeight':'bold',
                                                  'color': colors['text'],
                                                  'textAlign':'center'},
                                    style_cell={
                                        'backgroundColor': 'rgb(50, 50, 50)',
                                        'color': colors['text'],
                                        'textAlign':'center'
                                    },
                                    page_action="native",
                                    page_current= 2,
                                    page_size= 4,
                                  
                                )
                            ],
                            )
                    ],
                    style={'width': '100%', 'display': 'inline-block','marginTop':'10px'},
                    ),
                    
                      ]
           )
        
      ])
    ])
         
            
 
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=True,dev_tools_ui=False)

Writing Sarit_app.py


In [ ]:
# Below command generate ngrok link... open link that link after program execution done
get_ipython().system_raw('./ngrok http 8050 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

!python Sarit_app.py





Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range
<IPython.core.display.Javascript object>
<IPython.core.display.Javascript object>
<IPython.core.display.Javascript object>
/usr/local/lib/python3.6/dist-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
<IPython.core.display.Javascript object>
Henry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)
<IPython.core.display.Javascript object>
<class 'pandas.core.frame.DataFrame'>
<IPython.core.display.Javascript objec